In [1]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import torch
from sklearn.manifold import TSNE

from models.dec import train_end_to_end

DATA_ROOT = Path('processed_data')
PREPROCESSING_METHOD = 'raw'
FOLD_ID = 0

X_train = np.load(DATA_ROOT / PREPROCESSING_METHOD / f"fold_{FOLD_ID}" / 'X_train.npy')
y_train = np.load(DATA_ROOT / PREPROCESSING_METHOD / f"fold_{FOLD_ID}" / 'y_train.npy')
# Shuffle both arrays
p = np.random.permutation(len(X_train))
X_train, y_train = X_train[p], y_train[p]

X_val = np.load(DATA_ROOT / PREPROCESSING_METHOD / f"fold_{FOLD_ID}" / 'X_val.npy')
y_val = np.load(DATA_ROOT / PREPROCESSING_METHOD / f"fold_{FOLD_ID}" / 'y_val.npy')
# Shuffle both arrays
p = np.random.permutation(len(X_val))
X_val, y_val = X_val[p], y_val[p]

print(f"Training dataset: {X_train.shape}; Validation dataset: {X_val.shape}")

Training dataset: (5668, 200); Validation dataset: (13250, 200)


In [2]:
latent_dim = 32
layer_sizes = [512, 128]
n_clusters = len(np.unique(y_train))
dec_cluster_init = 'kmeans'
device = 'cuda'

pretrain_args = {
    'batch_size': 128,
    'n_epochs': 10,
    'lr': 1E-3,
    'weight_decay': 2E-5,
    'noise_std': .0
}

dec_training_args = {
    'tol': 1E-3,
    'update_interval': 300,
    'batch_size': 512,
    'lr': 1E-3,
    'weight_decay': 2E-5,
}

dec_model = train_end_to_end(X_train, X_val=None, n_clusters=n_clusters, dec_clustering_init=dec_cluster_init,
                             latent_dim=latent_dim, layer_sizes=layer_sizes,
                             pretrain_args=pretrain_args, dec_train_args=dec_training_args,
                             verbose=True, device=device)

Starting pretraining!
Pretraining using device: cuda
Epoch 1/10
Training time: 0.32s; Training loss: 149.02889;

Epoch 2/10
Training time: 0.07s; Training loss: 100.54580;

Epoch 3/10
Training time: 0.07s; Training loss: 86.23560;

Epoch 4/10
Training time: 0.07s; Training loss: 74.80923;

Epoch 5/10
Training time: 0.07s; Training loss: 66.99379;

Epoch 6/10
Training time: 0.07s; Training loss: 61.77166;

Epoch 7/10
Training time: 0.07s; Training loss: 57.72183;

Epoch 8/10
Training time: 0.07s; Training loss: 54.45939;

Epoch 9/10
Training time: 0.07s; Training loss: 51.68223;

Epoch 10/10
Training time: 0.07s; Training loss: 49.82512;

Pretraining finished.
Training DEC!
Training using device: cuda
Initializing using KMeans!
Iter 0 - Loss: 0.124050 - Delta: 0.433594
Iter 300 - Loss: 0.002017 - Delta: 0.000000
DEC trained.


In [3]:
z = encoder(torch.from_numpy(X_val).to(torch.float32).to(device)).detach().cpu().numpy()
tsne = TSNE(n_components=2)
z_enc = tsne.fit_transform(z)

colors = plt.cm.rainbow(np.linspace(0, 1, len(np.unique(y_val))))
for class_id in np.unique(y_val):
    plt.scatter(z_enc[y_val == class_id, 0], z_enc[y_val == class_id, 1], c=colors[class_id], s=5, alpha=.1)

plt.show()

NameError: name 'encoder' is not defined